# Fluid dynamics with LBM

Code based on the python implementation from: https://palabos.unige.ch/get-started/lattice-boltzmann/lattice-boltzmann-sample-codes-various-other-programming-languages/

Only the obstacles part is modified.

In [ ]:
#!/usr/bin/python
# Copyright (C) 2013 FlowKit Ltd, Lausanne, Switzerland
# E-mail contact: contact@flowkit.com
#
# This program is free software: you can redistribute it and/or
# modify it under the terms of the GNU General Public License, either
# version 3 of the License, or (at your option) any later version.

#
# 2D flow around a cylinder
#

from numpy import *; from numpy.linalg import *
import matplotlib.pyplot as plt; from matplotlib import cm
###### Flow definition #########################################################
maxIter = 200000 # Total number of time iterations.
Re      = 220.0  # Reynolds number.
nx = 520; ny = 260; ly=ny-1.0; q = 9 # Lattice dimensions and populations.
cx = nx/4; cy=ny/2; r=ny/9;          # Coordinates of the cylinder.
uLB     = 0.04                       # Velocity in lattice units.
nulb    = uLB*r/Re; omega = 1.0 / (3.*nulb+0.5); # Relaxation parameter.

###### Lattice Constants #######################################################
c = array([(x,y) for x in [0,-1,1] for y in [0,-1,1]]) # Lattice velocities.
t = 1./36. * ones(q)                                   # Lattice weights.
t[asarray([norm(ci)<1.1 for ci in c])] = 1./9.; t[0] = 4./9.
noslip = [c.tolist().index((-c[i]).tolist()) for i in range(q)] 
i1 = arange(q)[asarray([ci[0]<0  for ci in c])] # Unknown on right wall.
i2 = arange(q)[asarray([ci[0]==0 for ci in c])] # Vertical middle.
i3 = arange(q)[asarray([ci[0]>0  for ci in c])] # Unknown on left wall.

###### Function Definitions ####################################################
sumpop = lambda fin: sum(fin,axis=0) # Helper function for density computation.
def equilibrium(rho,u):              # Equilibrium distribution function.
    cu   = 3.0 * dot(c,u.transpose(1,0,2))
    usqr = 3./2.*(u[0]**2+u[1]**2)
    feq = zeros((q,nx,ny))
    for i in range(q): feq[i,:,:] = rho*t[i]*(1.+cu[i]+0.5*cu[i]**2-usqr)
    return feq

r=10
###### Setup: cylindrical obstacle and velocity inlet with perturbation ########
obstacle1 = fromfunction(lambda x,y: (x-100)**2+(y-90)**2<r**2, (nx,ny))
obstacle2 = fromfunction(lambda x,y: (x-140)**2+(y-90)**2<r**2, (nx,ny))
obstacle3 = fromfunction(lambda x,y: (x-180)**2+(y-90)**2<r**2, (nx,ny))
obstacle4 = fromfunction(lambda x,y: (x-100)**2+(y-130)**2<r**2, (nx,ny))
obstacle5 = fromfunction(lambda x,y: (x-140)**2+(y-130)**2<r**2, (nx,ny))
obstacle6 = fromfunction(lambda x,y: (x-180)**2+(y-130)**2<r**2, (nx,ny))
obstacle7 = fromfunction(lambda x,y: (x-100)**2+(y-170)**2<r**2, (nx,ny))
obstacle8 = fromfunction(lambda x,y: (x-140)**2+(y-170)**2<r**2, (nx,ny))
obstacle9 = fromfunction(lambda x,y: (x-180)**2+(y-170)**2<r**2, (nx,ny))
vel = fromfunction(lambda d,x,y: (1-d)*uLB*(1.0+1e-4*sin(y/ly*2*pi)),(2,nx,ny))
feq = equilibrium(1.0,vel); fin = feq.copy()

###### Main time loop ##########################################################
for time in range(maxIter):
    fin[i1,-1,:] = fin[i1,-2,:] # Right wall: outflow condition.
    rho = sumpop(fin)           # Calculate macroscopic density and velocity.
    u = dot(c.transpose(), fin.transpose((1,0,2)))/rho

    u[:,0,:] =vel[:,0,:] # Left wall: compute density from known populations.
    rho[0,:] = 1./(1.-u[0,0,:]) * (sumpop(fin[i2,0,:])+2.*sumpop(fin[i1,0,:]))

    feq = equilibrium(rho,u) # Left wall: Zou/He boundary condition.
    fin[i3,0,:] = fin[i1,0,:] + feq[i3,0,:] - fin[i1,0,:]
    fout = fin - omega * (fin - feq)  # Collision step.
    for i in range(q): fout[i,obstacle1] = fin[noslip[i],obstacle1]
    for i in range(q): fout[i,obstacle2] = fin[noslip[i],obstacle2]
    for i in range(q): fout[i,obstacle3] = fin[noslip[i],obstacle3]
    for i in range(q): fout[i,obstacle4] = fin[noslip[i],obstacle4]
    for i in range(q): fout[i,obstacle5] = fin[noslip[i],obstacle5]
    for i in range(q): fout[i,obstacle6] = fin[noslip[i],obstacle6]
    for i in range(q): fout[i,obstacle7] = fin[noslip[i],obstacle7]
    for i in range(q): fout[i,obstacle8] = fin[noslip[i],obstacle8]
    for i in range(q): fout[i,obstacle9] = fin[noslip[i],obstacle9]
    for i in range(q): # Streaming step.
        fin[i,:,:] = roll(roll(fout[i,:,:],c[i,0],axis=0),c[i,1],axis=1)
 
    if (time%100==0): # Visualization
        plt.clf(); plt.imshow(sqrt(u[0]**2+u[1]**2).transpose(),cmap=cm.jet)
        plt.xlabel('x')
        plt.ylabel('y')
        cbar = plt.colorbar()
        cbar.set_label('|u| (au)', rotation=90)
        plt.savefig("jetflow."+str(time/100).zfill(4)+".png",dpi=300)
        

